In [27]:
import pandas as pd

In [28]:
df_historical_data = pd.read_csv('/content/sample_data/fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('/content/sample_data/fifa_worldcup_2022_fixtures.csv')
df_missing_data = pd.read_csv('/content/sample_data/fifa_worldcup_missing_data.csv')

In [29]:
# cleaning df_fixture

df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

In [30]:
#cleaning df_missing_data and adding it to df_historical_data

#null data
df_missing_data[df_missing_data['home'].isnull()]

# dropping null data
df_missing_data.dropna(inplace = True)

# concatinating both the dfs and clean
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
827,Brazil,2–0,Costa Rica,2018
828,Serbia,1–2,Switzerland,2018
829,Serbia,0–2,Brazil,2018
831,Germany,0–1,Mexico,2018


In [31]:
# deleting match with walk over
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

# cleanning score and home/away columns
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True) # used to remove the word Extra Time
df_historical_data['home'] = df_historical_data['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_historical_data['away'] = df_historical_data['away'].str.strip()

# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam',
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']

df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
827,Brazil,Costa Rica,2018,2,0,2
828,Serbia,Switzerland,2018,1,2,3
829,Serbia,Brazil,2018,0,2,2
831,Germany,Mexico,2018,0,1,1


In [33]:
# exporting the cleaned dataframes
df_historical_data.to_csv('cleaned_fifa_worldcup_matches.csv', index=False)
df_fixture.to_csv('cleaned_fifa_worldcup_2022_fixtures.csv', index=False)